In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import torch
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data, Batch
from tqdm import tqdm
import torch.nn.functional as F


In [2]:
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Apple GPU")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA GPU")
else:
    device = torch.device('cpu')

Apple GPU


In [3]:
def getData(path):
    train_file = np.load(path+"/train.npz")
    train_data = train_file['data']
    test_file = np.load(path+"/test_input.npz")
    test_data = test_file['data']
    print(f"Training Data's shape is {train_data.shape} and Test Data's is {test_data.shape}")
    return train_data, test_data
trainData, testData = getData("./data/")

Training Data's shape is (10000, 50, 110, 6) and Test Data's is (2100, 50, 50, 6)


In [ ]:
class WindowedNormalizedDataset(Dataset):
    def __init__(self, data, scale=10.0):
        self.data = data  
        self.scale = scale

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        scene = self.data[idx].copy()  

        presence = (scene[..., 0] != 0) | (scene[..., 1] != 0)  

        origin = scene[0, 49].copy()
        tx, ty, _, _, theta, _ = origin

        cos_theta = np.cos(-theta)
        sin_theta = np.sin(-theta)

        normalized_scene = np.zeros((50, 110, 7), dtype=np.float32)

        # --- Feature Normalization + Scaling ---
        x = scene[..., 0] - tx
        y = scene[..., 1] - ty
        x_n = x * cos_theta - y * sin_theta
        y_n = x * sin_theta + y * cos_theta
        normalized_scene[..., 0] = x_n / self.scale
        normalized_scene[..., 1] = y_n / self.scale

        vx = scene[..., 2]
        vy = scene[..., 3]
        vx_n = vx * cos_theta - vy * sin_theta
        vy_n = vx * sin_theta + vy * cos_theta
        normalized_scene[..., 2] = vx_n / self.scale
        normalized_scene[..., 3] = vy_n / self.scale

        heading = scene[..., 4]
        normalized_heading = heading - theta
        normalized_heading = (normalized_heading + np.pi) % (2 * np.pi) - np.pi
        normalized_scene[..., 4] = normalized_heading

        normalized_scene[..., 5] = scene[..., 5]  
        normalized_scene[..., 6] = presence.astype(np.float32)

        missing_mask = np.expand_dims(~presence, -1)
        normalized_scene[..., :5] = np.where(missing_mask, 0, normalized_scene[..., :5])

        X = normalized_scene[:, :50, :]
        ego_future = normalized_scene[0, 50:]
        Y = np.zeros((60, 3), dtype=np.float32)
        Y[:, :2] = ego_future[:, :2]
        Y[:, 2] = ego_future[:, 6]

        return (
            torch.tensor(X, dtype=torch.float32),
            torch.tensor(Y, dtype=torch.float32),
            torch.tensor(origin, dtype=torch.float32)
        )


In [ ]:
class WindowedNormalizedDataset(Dataset):
    def __init__(self, data, scale=10.0):
        self.data = data
        self.scale = scale

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        scene = self.data[idx].copy()
        presence = (scene[..., 0] != 0) | (scene[..., 1] != 0)

        origin = scene[0, 49].copy()
        tx, ty, _, _, theta, _ = origin
        cos_theta = np.cos(-theta)
        sin_theta = np.sin(-theta)

        normalized_scene = np.zeros((50, 110, 13), dtype=np.float32)

        # Positions
        x = scene[..., 0] - tx
        y = scene[..., 1] - ty
        x_n = x * cos_theta - y * sin_theta
        y_n = x * sin_theta + y * cos_theta
        normalized_scene[..., 0] = x_n / self.scale
        normalized_scene[..., 1] = y_n / self.scale
        
        # Velocities
        vx = scene[..., 2]
        vy = scene[..., 3]
        vx_n = vx * cos_theta - vy * sin_theta
        vy_n = vx * sin_theta + vy * cos_theta
        normalized_scene[..., 2] = vx_n / self.scale
        normalized_scene[..., 3] = vy_n / self.scale
        
        # Heading
        heading = scene[..., 4]
        normalized_heading = heading - theta
        normalized_heading = (normalized_heading + np.pi) % (2 * np.pi) - np.pi
        normalized_scene[..., 4] = normalized_heading
        
        # Agent Type & Presence
        normalized_scene[..., 5] = scene[..., 5]
        normalized_scene[..., 6] = presence.astype(np.float32)

        # --- New Feature 1: Acceleration Magnitude (Index 7) ---
        vx_norm = normalized_scene[..., 2]
        vy_norm = normalized_scene[..., 3]
        ax = np.zeros_like(vx_norm)
        ay = np.zeros_like(vy_norm)
        ax[:, 1:] = (vx_norm[:, 1:] - vx_norm[:, :-1]) * 10.0  # 10Hz data
        ay[:, 1:] = (vy_norm[:, 1:] - vy_norm[:, :-1]) * 10.0
        acc_mag = np.sqrt(ax**2 + ay**2)
        normalized_scene[..., 7] = acc_mag

        # --- New Feature 2: Relative Heading to Ego (Index 8) ---
        ego_heading = normalized_scene[0, :, 4]  
        agent_heading = normalized_scene[..., 4] 
        relative_heading = agent_heading - ego_heading[np.newaxis, :]
        relative_heading = (relative_heading + np.pi) % (2 * np.pi) - np.pi
        normalized_scene[..., 8] = relative_heading

        # --- New Feature 3: Interaction Criticality (Index 9) ---
        dist_to_ego = np.linalg.norm(scene[..., :2] - scene[0, :, :2], axis=-1)
        normalized_scene[..., 10] = dist_to_ego / self.scale
        
        # Compute time-to-collision proxy
        rel_vel = np.linalg.norm(scene[..., 2:4] - scene[0, :, 2:4], axis=-1)
        ttc_proxy = np.zeros_like(dist_to_ego)
        valid_mask = (dist_to_ego > 0.1) & (rel_vel > 0.1)
        ttc_proxy[valid_mask] = dist_to_ego[valid_mask] / rel_vel[valid_mask]
        normalized_scene[..., 9] = np.clip(ttc_proxy, 0, 10)  # Cap at 10s

        # --- Existing Features ---
        normalized_scene[..., 11] = np.sqrt(vx**2 + vy**2) / self.scale  
        normalized_scene[..., 12] = (np.arange(50)[:, None] == 0).astype(np.float32)  
        
        # Mask invalid timesteps
        missing_mask = np.expand_dims(~presence, -1)
        normalized_scene[..., :13] = np.where(missing_mask, 0, normalized_scene[..., :14])

        # Inputs: first 50 timesteps
        X = normalized_scene[:, :50, :]  

        # Target: ego agent's future positions and presence
        ego_future = normalized_scene[0, 50:]
        Y = np.zeros((60, 3), dtype=np.float32)
        Y[:, :2] = ego_future[:, :2]
        Y[:, 2] = ego_future[:, 6]  

        return (
            torch.tensor(X, dtype=torch.float32),
            torch.tensor(Y, dtype=torch.float32),
            torch.tensor(origin, dtype=torch.float32)
        )

In [ ]:
class WindowedNormalizedTestDataset(Dataset):
    def __init__(self, data, scale):
        self.data = data  
        self.scale = scale

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        scene = self.data[idx].copy()  
        
        # Create presence flag: 1 if agent present, 0 if missing (both x and y are zero)
        presence = (scene[..., 0] != 0) | (scene[..., 1] != 0)  
        
        # Get reference values from ego vehicle (agent 0) at t=49
        origin = scene[0, 49].copy()  
        tx, ty, _, _, theta, _ = origin  
        
        # Compute rotation matrix for -theta (align ego heading to x-axis)
        cos_theta = np.cos(-theta)
        sin_theta = np.sin(-theta)
        
        # Initialize normalized scene with 7 features
        normalized_scene = np.zeros((50, 50, 7), dtype=np.float32)
        
        # --- Feature Normalization ---
        x = scene[..., 0] - tx
        y = scene[..., 1] - ty
        normalized_scene[..., 0] = x * cos_theta - y * sin_theta  
        normalized_scene[..., 1] = x * sin_theta + y * cos_theta  
        normalized_scene[..., 0] = normalized_scene[..., 0]/ self.scale
        normalized_scene[..., 1] = normalized_scene[..., 1]/ self.scale
        
        # Velocities
        vx = scene[..., 2]
        vy = scene[..., 3]
        normalized_scene[..., 2] = vx * cos_theta - vy * sin_theta  
        normalized_scene[..., 3] = vx * sin_theta + vy * cos_theta  
        normalized_scene[..., 2] = normalized_scene[..., 2] / self.scale
        normalized_scene[..., 3] = normalized_scene[..., 3] / self.scale
        
        # Heading (normalize relative to ego and wrap to [-π, π])
        heading = scene[..., 4]
        normalized_heading = heading - theta
        normalized_heading = (normalized_heading + np.pi) % (2 * np.pi) - np.pi
        normalized_scene[..., 4] = normalized_heading
        
        # Agent type (unchanged) and presence flag
        normalized_scene[..., 5] = scene[..., 5]  
        normalized_scene[..., 6] = presence.astype(np.float32)  
        
        # Reset features for missing agents (where presence=0)
        missing_mask = np.expand_dims(~presence, -1)  
        normalized_scene[..., :5] = np.where(missing_mask, 0, normalized_scene[..., :5])
        
        # Split into input (0-49) and target (50-109)
        X = normalized_scene[:, :50, :]  
        
        return (
            torch.tensor(X, dtype=torch.float32),
            torch.tensor(origin, dtype=torch.float32)
        )

def denormalize_ego(predicted, origin):
    """
    Convert normalized ego predictions back to global coordinates
    predicted: (..., 2) tensor of normalized [x, y] positions
    origin: (6,) tensor of ego's reference state at t=49
    """
    tx, ty, _, _, theta, _ = origin
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    
    x_rot = predicted[..., 0] * cos_theta - predicted[..., 1] * sin_theta
    y_rot = predicted[..., 0] * sin_theta + predicted[..., 1] * cos_theta
    
    x_global = x_rot + tx
    y_global = y_rot + ty
    
    return torch.stack([x_global, y_global], dim=-1)

In [ ]:
class WindowedNormalizedTestDataset(Dataset):
    def __init__(self, data, scale=10.0):
        self.data = data
        self.scale = scale

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        scene = self.data[idx].copy()
        presence = (scene[..., 0] != 0) | (scene[..., 1] != 0)

        origin = scene[0, 49].copy()
        tx, ty, _, _, theta, _ = origin
        cos_theta = np.cos(-theta)
        sin_theta = np.sin(-theta)

        normalized_scene = np.zeros((50, 50, 13), dtype=np.float32)

        # Positions
        x = scene[..., 0] - tx
        y = scene[..., 1] - ty
        x_n = x * cos_theta - y * sin_theta
        y_n = x * sin_theta + y * cos_theta
        normalized_scene[..., 0] = x_n / self.scale
        normalized_scene[..., 1] = y_n / self.scale
        
        # Velocities
        vx = scene[..., 2]
        vy = scene[..., 3]
        vx_n = vx * cos_theta - vy * sin_theta
        vy_n = vx * sin_theta + vy * cos_theta
        normalized_scene[..., 2] = vx_n / self.scale
        normalized_scene[..., 3] = vy_n / self.scale
        
        # Heading
        heading = scene[..., 4]
        normalized_heading = heading - theta
        normalized_heading = (normalized_heading + np.pi) % (2 * np.pi) - np.pi
        normalized_scene[..., 4] = normalized_heading
        
        # Agent Type & Presence
        normalized_scene[..., 5] = scene[..., 5]
        normalized_scene[..., 6] = presence.astype(np.float32)

        # --- New Feature 1: Acceleration Magnitude (Index 7) ---
        vx_norm = normalized_scene[..., 2]
        vy_norm = normalized_scene[..., 3]
        ax = np.zeros_like(vx_norm)
        ay = np.zeros_like(vy_norm)
        ax[:, 1:] = (vx_norm[:, 1:] - vx_norm[:, :-1]) * 10.0  # 10Hz data
        ay[:, 1:] = (vy_norm[:, 1:] - vy_norm[:, :-1]) * 10.0
        acc_mag = np.sqrt(ax**2 + ay**2)
        normalized_scene[..., 7] = acc_mag

        # --- New Feature 2: Relative Heading to Ego (Index 8) ---
        ego_heading = normalized_scene[0, :, 4]  
        agent_heading = normalized_scene[..., 4]  
        relative_heading = agent_heading - ego_heading[np.newaxis, :]
        relative_heading = (relative_heading + np.pi) % (2 * np.pi) - np.pi
        normalized_scene[..., 8] = relative_heading

        # --- New Feature 3: Interaction Criticality (Index 9) ---
        dist_to_ego = np.linalg.norm(scene[..., :2] - scene[0, :, :2], axis=-1)
        normalized_scene[..., 10] = dist_to_ego / self.scale
        
        # Compute time-to-collision proxy
        rel_vel = np.linalg.norm(scene[..., 2:4] - scene[0, :, 2:4], axis=-1)
        ttc_proxy = np.zeros_like(dist_to_ego)
        valid_mask = (dist_to_ego > 0.1) & (rel_vel > 0.1)
        ttc_proxy[valid_mask] = dist_to_ego[valid_mask] / rel_vel[valid_mask]
        normalized_scene[..., 9] = np.clip(ttc_proxy, 0, 10)

        # --- Existing Features ---
        normalized_scene[..., 11] = np.sqrt(vx**2 + vy**2) / self.scale  
        normalized_scene[..., 12] = (np.arange(50)[:, None] == 0).astype(np.float32)  

        # Mask invalid timesteps
        missing_mask = np.expand_dims(~presence, -1)
        normalized_scene[..., :13] = np.where(missing_mask, 0, normalized_scene[..., :14])

        # Inputs: first 50 timesteps
        X = normalized_scene[:, :50, :]  

        return (
            torch.tensor(X, dtype=torch.float32),
            torch.tensor(origin, dtype=torch.float32)
        )

In [ ]:
def denormalize_ego_batch(predicted, origin, scale=10.0):
    """
    Convert batch of normalized (and scaled) ego predictions back to global coordinates.

    predicted: (B, ..., 2) tensor of normalized [x, y] positions
    origin: (B, 6) tensor of ego's reference state at t=49
    Returns:
        (B, ..., 2) tensor of global [x, y] positions
    """
    tx = origin[:, 0]  
    ty = origin[:, 1]  
    theta = origin[:, 4]  

    cos_theta = torch.cos(theta)
    sin_theta = torch.sin(theta)

    # Expand for broadcasting
    while len(cos_theta.shape) < len(predicted.shape) - 1:
        cos_theta = cos_theta.unsqueeze(1)
        sin_theta = sin_theta.unsqueeze(1)
        tx = tx.unsqueeze(1)
        ty = ty.unsqueeze(1)

    x = predicted[..., 0] * scale
    y = predicted[..., 1] * scale

    x_rot = x * cos_theta - y * sin_theta
    y_rot = x * sin_theta + y * cos_theta

    x_global = x_rot + tx
    y_global = y_rot + ty

    return torch.stack([x_global, y_global], dim=-1)


In [22]:
trainData[1, 0, 49, :], trainData[1, 0, 50, :]

(array([ 3.16906469e+03,  1.68248551e+03,  5.46145515e+00, -5.85380650e+00,
        -8.22467566e-01,  0.00000000e+00]),
 array([ 3.16959927e+03,  1.68191109e+03,  5.35655550e+00, -5.75120145e+00,
        -8.22600550e-01,  0.00000000e+00]))

In [23]:
data = WindowedNormalizedDataset(trainData)
X, Y, origin = data.__getitem__(1)
X[0, 49, :], Y[0, :], origin.shape

(tensor([0.0000, 0.0000, 0.8006, 0.0019, 0.0000, 0.0000, 1.0000, 0.0572, 0.0000,
         0.0000, 0.0000, 0.8006, 1.0000]),
 tensor([7.8468e-02, 9.1270e-05, 1.0000e+00]),
 torch.Size([6]))

In [24]:
# x, y = denormalize_ego(Y[0, :2], origin)
# x, y

In [ ]:
import torch
import torch.nn as nn

class TrajectoryTransformer(nn.Module):
    def __init__(self, input_dim=650, model_dim=256, num_heads=8, num_layers=6, dropout=0.1, pred_len=60, num_agents=50):
        super().__init__()
        self.model_dim = model_dim
        self.pred_len = pred_len
        self.num_agents = num_agents
        
        self.trajectory_encoder = nn.Sequential(
            nn.Linear(input_dim, model_dim),
            nn.LayerNorm(model_dim),
            nn.ReLU(),
            nn.Linear(model_dim, model_dim),
            nn.LayerNorm(model_dim),
            nn.ReLU(),
            nn.Linear(model_dim, model_dim),
            nn.LayerNorm(model_dim),
            nn.ReLU()
        )
        
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=model_dim, 
                nhead=num_heads, 
                dropout=dropout, 
                batch_first=True
            ),
            num_layers=num_layers
        )
        
        self.output_fcpre = nn.Linear(model_dim, model_dim)  
        self.output_fc = nn.Linear(model_dim, pred_len * 2)  
    
    def forward(self, x):
        B, N, T, Ft = x.shape
        x = x.view(B, N, T * Ft)  
        agent_tokens = self.trajectory_encoder(x)  
        encoded_tokens = self.transformer_encoder(agent_tokens)  
        ego_token = encoded_tokens[:, 0, :]  
        output = F.relu(self.output_fcpre(ego_token))  
        output = self.output_fc(output)  
        output = output.view(B, self.pred_len, 2) 
        
        return output

model = TrajectoryTransformer()
x = torch.randn(1, 50, 50, 13)  
out = model(x)
print(f"Input shape: {x.shape}")
print(f"Output shape: {out.shape}")  

print(f"\nModel parameters: {sum(p.numel() for p in model.parameters()):,}")

Input shape: torch.Size([1, 50, 50, 13])
Output shape: torch.Size([1, 60, 2])

Model parameters: 8,286,840


In [26]:
model = TrajectoryTransformer().to(device=device)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 8286840


In [ ]:
np.random.seed(42)
num_samples = trainData.shape[0]
indices = np.random.permutation(num_samples)
split_index = int(0.9 * num_samples)
train_idx, val_idx = indices[:split_index], indices[split_index:]

train_data = trainData[train_idx]
val_data = trainData[val_idx]

print("Train shape:", train_data.shape)
print("Validation shape:", val_data.shape)

Train shape: (9000, 50, 110, 6)
Validation shape: (1000, 50, 110, 6)


In [28]:
trainTensor = WindowedNormalizedDataset(train_data)
testTensor = WindowedNormalizedDataset(val_data)
train_dataloader = DataLoader(trainTensor, batch_size=128, shuffle=True)
val_dataloader = DataLoader(testTensor, batch_size=128, shuffle=False)

In [29]:
import torch
import torch.nn as nn

class WeightedMSELoss(nn.Module):
    def __init__(self, weights):
        super(WeightedMSELoss, self).__init__()
        self.weights = weights

    def forward(self, input, target):
        loss = (input - target) ** 2
        weighted_loss = loss * self.weights
        return weighted_loss.mean()

feature_weights = torch.tensor([1.0, 1.0, 0.1, 0.1, 0.01, 0.01], device=device)


In [ ]:
# export PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0
#  train MSE 0.0130673476 | train val MSE 0.0830958160 | val MAE 4.7582098953 | val MSE 8.3095822744

torch.cuda.empty_cache()

best_model = torch.load("./models/final/best_model.pt")
model.load_state_dict(best_model)

epochs = 1000
lossFn = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.25)
best_val_loss = 0.0830958160 # float('inf')
best_train_loss = 0.0130673476 # float('inf')
position_scale = 1.0
velocity_scale = 1.0
all_losses = {
    'training_mse_loss':[],
    'validation_mse_loss':[],
    'true_mse':[],
    'true_mae':[]
}

for each_epoch in range(epochs):
    model.train()
    runningLoss = 0.0
    loop = tqdm(train_dataloader, desc=f"Epoch [{each_epoch+1}/{epochs}]")
    
    for batchX, batchY, origin in loop:
        batchX = batchX.to(device)
        batchY = batchY.to(device)
        origin = origin.to(device)

        
        pred = model(batchX)  
        
        loss = lossFn(pred[..., :2], batchY[..., :2]).to(device)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        runningLoss += loss.item()        
    
    model.eval()
    val_loss = 0
    val_mae = 0
    val_mse = 0
    
    with torch.no_grad():
        for batchX, batchY, origin in loop:
            batchX = batchX.to(device)
            batchY = batchY.to(device)
            origin = origin.to(device)

            
            pred = model(batchX)  
            
            loss = lossFn(pred[..., :2], batchY[..., :2]).to(device)
            unnorm_pred = denormalize_ego_batch(pred, origin)
            unnorm_true = denormalize_ego_batch(batchY, origin)


            
            val_loss += loss.item()
            val_mae += nn.L1Loss()(unnorm_pred[..., :2], unnorm_true[..., :2]).item()
            val_mse += nn.MSELoss()(unnorm_pred[..., :2], unnorm_true[..., :2]).item()
    # break
    train_loss = runningLoss/len(train_dataloader)
    val_loss /= len(val_dataloader)
    val_mae /= len(val_dataloader)
    val_mse /= len(val_dataloader)
    
    all_losses["training_mse_loss"].append(train_loss)
    all_losses["validation_mse_loss"].append(val_loss)
    all_losses["true_mse"].append(val_mse)
    all_losses["true_mae"].append(val_mae)
    
    loop.write(f" train MSE {train_loss:.10f} | train val MSE {val_loss:.10f} | val MAE {val_mae:.10f} | val MSE {val_mse:.10f}")
    scheduler.step()
    
    if train_loss < best_train_loss and val_loss < best_val_loss: 
        best_val_loss = val_loss
        best_train_loss = train_loss
        no_improvement = 0
        torch.save(model.state_dict(), "./models/final/best_model.pt")
        loop.write(f" model Saved")
    torch.cuda.empty_cache()


Epoch [1/1000]: 100%|██████████| 71/71 [00:25<00:00,  2.73it/s]


 train MSE 0.0499150728 | train val MSE 0.2085464783 | val MAE 7.5795354098 | val MSE 20.8546437919


Epoch [2/1000]: 100%|██████████| 71/71 [00:26<00:00,  2.72it/s]


 train MSE 0.0311552232 | train val MSE 0.2784796467 | val MAE 8.8686763421 | val MSE 27.8479639292


Epoch [3/1000]: 100%|██████████| 71/71 [00:25<00:00,  2.73it/s]


...

 train MSE 0.0055931962 | train val MSE 0.0337440327 | val MAE 3.1509329565 | val MSE 3.3744038045


Epoch [132/1000]: 100%|██████████| 71/71 [00:26<00:00,  2.73it/s]


 train MSE 0.0055336726 | train val MSE 0.0336720448 | val MAE 3.1481382884 | val MSE 3.3672049008


Epoch [133/1000]:  39%|███▉      | 28/71 [00:11<00:17,  2.45it/s]


KeyboardInterrupt: 

In [ ]:
test_dataset = WindowedNormalizedTestDataset(testData)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


best_model = torch.load("./models/final/best_model.pt")
model = model = TrajectoryTransformer().to(device=device)
# model = model = TrajectoryTransformerPlus().to(device=device)


model.load_state_dict(best_model)
model.eval()

pred_list = []
with torch.no_grad():
    for batchX, origin in test_loader:
        batchX = batchX.to(device)
        batchY = batchY.to(device)
        origin = origin.to(device)

        
        pred = model(batchX)  
        
        unnorm_pred = denormalize_ego_batch(pred[..., :2], origin)
        # print(unnorm_pred.shape)
        pred_list.append(unnorm_pred.cpu().numpy())
        # print(len(pred))
        

pred_list = np.concatenate(pred_list, axis=0)  
pred_output = pred_list.reshape(-1, 2)  
output_df = pd.DataFrame(pred_output, columns=['x', 'y'])
output_df.index.name = 'index'
output_df.to_csv('./models/modelI/testTransFormer.csv', index=True)

In [ ]:
#  train MSE 0.0058623942 | train val MSE 0.0470800689 | val MAE 3.5586496554 | val MSE 4.7080054618  --- Test: 8.5
#  train MSE 0.0027955156 | train val MSE 0.0230247062 | val MAE 2.6988429800 | val MSE 2.3024725225 ---- Test: 8.43
